In [1]:
from numba import jit, njit, vectorize,config
import cProfile

In [2]:
import matplotlib.pyplot as plt
import seaborn as sbn
import pandas as pd
import numpy as np
import random

In [3]:
config.THREADING_LAYER = 'threadsafe'
sbn.set()

# A numba test for SSA

This notebook is an attemp to accelerate the algorithm presented previously in [Epoch_1](./Epoch_1.ipynb) thru numba decorators.   
The exploratory test will be deffenitevely large so you can consult a final version and analysis in INSET FINAL NUMBA to see results.

We will work at first with so called _Lotka Reaction_ system:
$$
\begin{eqnarray}
&\bar{X} + Y_{1}& \xrightarrow{\;\; C_{1} \;\; } 2Y_{1} \\
&Y_{1} + Y_{2}& \xrightarrow{\;\; C_{2} \;\; } 2Y_{2} \\
&Y_{2}& \xrightarrow{\;\; C_{3} \;\; } Z
\end{eqnarray}
$$

Where $\bar{X}$ denotes that the molecular population  is constant over time.

The set of propensity equations is:
$$
\begin{eqnarray}
a_{1}(\mathbf{Y}) &=& c_1 [\bar{X}][Y_{1}] = k[Y_{1}]\\
a_{2}(\mathbf{Y}) &=& c_2 [Y_{2}][Y_{1}] \\
a_{3}(\mathbf{Y}) &=& c_3 [Y_{2}]
\end{eqnarray}
$$
And the asociated state-change vectors are:
$$
\begin{eqnarray}
\nu_{1} &=& [1,0] \\
\nu_{2} &=& [-1,-1] \\
\nu_{3} &=& [0,-1]
\end{eqnarray}
$$

A more clearner version of SSA before develped:  

In [141]:
@vectorize()
def expo(s): return np.random.exponential(scale=1/s)

In [147]:
def SSA(x,t,a,v,t_max,seed = 42):
    """
    A toy version of the SSA described in the notebook
    
    Params
    ---------
    x : array_like
        A vector of states vectors, if len(x)>1 then is going to be used x[-1]
    t : array_like
        A vecttor of timesteps, t0 = t[-1]
    a : func
        The vectorial function form of all propensity functions
    v : array_like
        The state change vector or matrix
    t_max : float
        The condition time to stop
    seed : int
        Seed to secure reproducibility
        
    Returns
    ----------
    x : array_like
        x.shape = (m,n) where m is number if reactions and n the number of
        molecular species
        The whole system vector-population after a R_{i} in a time t_{i}
    t : array_like
        The time steps at which each reaction occurs
    """
    if seed!=None:
        np.random.seed(seed)
    idx = np.arange(len(v))
    while True:
        x0,t0 = x[-1],t[-1]
        
        #if np.sum(a(x0),axis=0).dtype == 'int64':
        #    a_vector = np.asarray([np.sum(a(x0),axis=0)])
        #else:
        a_vector = a(x0)

        a_0 = np.sum(a_vector)
        if a_0 == 0: #first stop condition
            break
            
        # Tau dist prob of generating function is a_{0}*exp(-tau*a_{0}) therefore the following is more stable than the later tau-generator
        #tau = np.random.exponential(scale=1/a_0)
        tau = expo(a_0)
        # the dist prob of generating function of j is a_{j}/a_{0}, a point probability so the following is more stable than the later j-generator
        j = np.random.choice(idx,p=a_vector/a_0)
                
        t = np.hstack((t,t0+tau))
        x = np.vstack((x,x0+v[j]))
        if t0+tau>=t_max:
            break
        
    return x,t

To reproduce results we're going to use $k=c_{1}[\bar{X}] = 10$, $c_{2}=0.01$ and $c_{3}=10$ with initial condition $\mathbf{Y_{0}}=[1000,1000]$

In [104]:
def R(Y,k,c2,c3):
    return np.asarray([k*Y[0],c2*Y[0]*Y[1], c3*Y[1]])

In [105]:
a = lambda y : R(y,k=10,c2=0.01,c3=10)
v = np.asarray([
    [1,0],
    [-1,1],
    [0,-1]
])

In [128]:
%%timeit -r 1
x,t = SSA(x=np.asarray([[1000,1000]]),t=[0],a=a,v=v,t_max=1)

3.56 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [126]:
%%timeit -r 1
x,t = SSA(x=np.asarray([[1000,1000]]),t=[0],a=a,v=v,t_max=1)

2.24 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Now, lets analize where the program is taking more time to run

In [146]:
cProfile.run('SSA(x=np.asarray([[1000,1000]]),t=[0],a=a,v=v,t_max=10)')

         20392529 function calls (19192969 primitive calls) in 146.884 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   299890    0.255    0.000    2.995    0.000 <__array_function__ internals>:2(atleast_1d)
   299890    0.255    0.000    1.934    0.000 <__array_function__ internals>:2(atleast_2d)
   599780    0.507    0.000  108.620    0.000 <__array_function__ internals>:2(concatenate)
   299890    0.298    0.000   41.070    0.000 <__array_function__ internals>:2(hstack)
   299890    0.298    0.000    4.223    0.000 <__array_function__ internals>:2(sum)
   299890    0.247    0.000   76.314    0.000 <__array_function__ internals>:2(vstack)
   299890    3.025    0.000    4.441    0.000 <ipython-input-104-474dae19171b>:1(R)
   299890    0.282    0.000    4.723    0.000 <ipython-input-105-9e47355c233e>:1(<lambda>)
        1    5.259    5.259  146.883  146.883 <ipython-input-145-9ef74a8d31b8>:1(SSA)
        1    0.001    0.0

In [148]:
cProfile.run('SSA(x=np.asarray([[1000,1000]]),t=[0],a=a,v=v,t_max=10)')

         20061752 function calls (18864036 primitive calls) in 127.966 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   299429    0.229    0.000    2.711    0.000 <__array_function__ internals>:2(atleast_1d)
   299429    0.221    0.000    1.743    0.000 <__array_function__ internals>:2(atleast_2d)
   598858    0.458    0.000   95.406    0.000 <__array_function__ internals>:2(concatenate)
   299429    0.258    0.000   35.827    0.000 <__array_function__ internals>:2(hstack)
   299429    0.272    0.000    3.682    0.000 <__array_function__ internals>:2(sum)
   299429    0.221    0.000   67.466    0.000 <__array_function__ internals>:2(vstack)
   299429    2.670    0.000    3.895    0.000 <ipython-input-104-474dae19171b>:1(R)
   299429    0.249    0.000    4.145    0.000 <ipython-input-105-9e47355c233e>:1(<lambda>)
        1    5.117    5.117  127.965  127.965 <ipython-input-147-3384a2cf93b0>:1(SSA)
        1    0.001    0.0

The most expensive methods are thos numpy releted. 

The attemp is to create an optimized function but general-purpose. Instead of give an _adhoc_ function, a function able to recive a function of propensity functions. 

In [7]:
@vectorize("float64(float64,float64)")
def add_vectors(a,b):
    return a+b

In [8]:
j,k = np.random.uniform(high=1000,size=1000),np.random.uniform(high=1000,size=1000)

In [9]:
%timeit j+k

1.48 µs ± 55.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [10]:
%timeit add_vectors(j,k)

1.39 µs ± 13.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Now, lets create an propensity function

In [11]:
@njit
def a_propensity(Y):
    k=10.0
    c2=0.01
    c3=10.0
    return np.array([k*Y[0],c2*Y[0]*Y[1], c3*Y[1]])

In [12]:
%timeit a([1000,1000])

2.67 µs ± 51.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [13]:
%timeit a_propensity(np.asarray([1000.0,1000.0]))

3.1 µs ± 56.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [14]:
@njit
def rand_choice_nb(arr, prob):
    """
    :param arr: A 1D numpy array of values to sample from.
    :param prob: A 1D numpy array of probabilities for the given samples.
    :return: A random sample from the given array with a given probability.
    """
    return arr[np.searchsorted(np.cumsum(prob), np.random.random(), side="right")]

In [80]:
@njit()
def SSA_jited(x,t,a,v,t_max,seed = 42):
    """
    A toy version of the SSA described in the notebook
    
    Params
    ---------
    x : array_like
        A vector of states vectors, if len(x)>1 then is going to be used x[-1]
    t : array_like
        A vecttor of timesteps, t0 = t[-1]
    a : afunc
        The vectorial function form of all propensity functions
    v : array_like
        The state change vector or matrix
    t_max : float
        The condition time to stop
    seed : int
        Seed to secure reproducibility
        
    Returns
    ----------
    x : array_like
        x.shape = (m,n) where m is number if reactions and n the number of
        molecular species
        The whole system vector-population after a R_{i} in a time t_{i}
    t : array_like
        The time steps at which each reaction occurs
    """
    np.random.seed(seed)

    idx = np.arange(len(v))
    while True:
        x0 = x[len(x)-1]
        t0 = t[len(x)-1]
        a_vector = a(x0)
        a_0 = np.sum(a_vector)
        if a_0 == 0: #first stop condition
            break
            
        # Tau dist prob of generating function is a_{0}*exp(-tau*a_{0}) therefore the following is more stable than the later tau-generator
        #tau = np.random.exponential(scale=1/a_0)
        tau = expo(a_0)
        # the dist prob of generating function of j is a_{j}/a_{0}, a point probability so the following is more stable than the later j-generator
        j = rand_choice_nb(arr=idx,prob=a_vector/a_0)
                
        t = np.append(t,t0+tau)
        #t = np.append((t,add_vectors(t0,tau).reshape(1)))
        dummy = add_vectors(x0,v[j])
        x = np.append(x,dummy.reshape(1,dummy.shape[0]),axis=0)
        #x = np.vstack((x,add_vectors(x0,v[j])))
        #print(t0+tau,a_vector,a_0,j)
        if t0+tau>=t_max:
            break
        
    return x,t

In [81]:
%timeit -r 1 SSA_jited(x=np.asarray([np.asarray([1000.0,1000.0])]),t=np.asarray([0.0]),a=a_propensity,v=v,t_max=4)

19 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [82]:
%timeit -r 1 SSA(x=np.asarray([[1000,1000]]),t=[0],a=a,v=v,t_max=4)

13.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [149]:
cProfile.run('SSA_jited(x=np.asarray([np.asarray([1000.0,1000.0])]),t=np.asarray([0.0]),a=a_propensity,v=v,t_max=10)')

         36 function calls in 152.372 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1  152.371  152.371  152.371  152.371 <ipython-input-80-f21a5649db58>:1(SSA_jited)
        1    0.000    0.000  152.372  152.372 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 _asarray.py:16(asarray)
        2    0.000    0.000    0.000    0.000 abstract.py:117(__hash__)
        2    0.000    0.000    0.000    0.000 abstract.py:48(_intern)
        2    0.000    0.000    0.000    0.000 abstract.py:60(__call__)
        2    0.000    0.000    0.000    0.000 abstract.py:92(__init__)
        2    0.000    0.000    0.000    0.000 dispatcher.py:664(__repr__)
        2    0.000    0.000    0.000    0.000 dispatcher.py:776(_numba_type_)
        2    0.000    0.000    0.000    0.000 functions.py:466(_store_object)
        2    0.000    0.000    0.000    0.000 functions.py:475(key)
        2    0.000    0.000    0.000    0

KeyboardInterrupt: 

In [87]:
cProfile.run('SSA(x=np.asarray([[1000,1000]]),t=[0],a=a,v=v,t_max=10)')

         10150165 function calls (9553097 primitive calls) in 23.911 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   149267    0.087    0.000    1.043    0.000 <__array_function__ internals>:2(atleast_1d)
   149267    0.076    0.000    0.638    0.000 <__array_function__ internals>:2(atleast_2d)
   298534    0.170    0.000   11.345    0.000 <__array_function__ internals>:2(concatenate)
   149267    0.106    0.000    5.723    0.000 <__array_function__ internals>:2(hstack)
   149267    0.096    0.000    1.331    0.000 <__array_function__ internals>:2(sum)
   149267    0.086    0.000    8.567    0.000 <__array_function__ internals>:2(vstack)
   149267    0.948    0.000    1.403    0.000 <ipython-input-5-474dae19171b>:1(R)
   149267    0.087    0.000    1.490    0.000 <ipython-input-6-9e47355c233e>:1(<lambda>)
        1    1.699    1.699   23.911   23.911 <ipython-input-72-4eac92b7d2c5>:1(SSA)
        1    0.000    0.000   23

In [112]:
@vectorize
def expo_v(s): return np.random.exponential(scale=1/s)

In [113]:
@njit()
def expo_j(s): return np.random.exponential(scale=1/s)

In [114]:
%timeit expo_v(2)

639 ns ± 34.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [115]:
%timeit expo_j(2)

202 ns ± 19.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [117]:
%timeit np.random.exponential(scale=1/2)

3.08 µs ± 192 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [118]:
rj = %timeit -qo expo_v(2)

In [119]:
r = %timeit -qo expo_j(2)

In [121]:
rj.worst>r.worst

True

In [123]:
rj.best>r.best

True